# HW11
## Problem 1: Computational cost of a convolutional layer

for each pixel: \\
$k*k*c*c$ \\
for the whole image,there are w*h pixels:  
$k*k*c*c*w*h$

## Problem 2: Neural networks with PyTorch

In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np

In [ ]:
def same_seeds(seed):
    # random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(42)

Load CIFAR10 train and test set

In [29]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4

trainset_0 = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset_0 = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


1. Reducing the size of the training and test accordingly

In [53]:
# car: 1
# dog: 5
# ship: 8
classes = {'car', 'dog', 'ship'}
class_mapping={1:0,5:1,8:2}

trainset = [ex for ex in trainset_0 if ex[1]==1 or ex[1]==5 or ex[1]==8]
testset = [ex for ex in testset_0 if ex[1]==1 or ex[1]==5 or ex[1]==8]

In [8]:
trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size,shuffle=False)

train_full = DataLoader(trainset, batch_size=len(trainset),shuffle=True)
train_X, train_y = next(iter(train_full))
train_X, train_y = train_X.to(device),train_y.to(device)
train_y = torch.tensor([class_mapping[label.item()] for label in train_y]).to(device)


test_full = DataLoader(testset, batch_size=len(testset),shuffle=True)
test_X, test_y = next(iter(test_full))
test_X, test_y = test_X.to(device),test_y.to(device)
test_y = torch.tensor([class_mapping[label.item()] for label in test_y]).to(device)

2. creat model

In [9]:
class Net(nn.Module):
    def __init__(self,inputdim):
        super(Net,self).__init__() # 3*32*32
        self.conv1 = nn.Conv2d(3,6,5) #input chanel, output chanel, kernel size
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84, 3)


    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1) # flatten all, except the batchsize
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net(3*32*32).to(device)

3. Optimizer and loss

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr = 1e-3, momentum=0.9)
max_epochs = 10

train_acc_best = 0
test_acc_best = 0
PATH="/model.pt"
for epoch in range(max_epochs):

    for i,data in enumerate(trainloader):
        optimizer.zero_grad()
        X,y = data
        X = X.to(device)
        y = y.to(device)
        y_mapped = torch.tensor([class_mapping[label.item()] for label in y]).to(device)
        # forward
        outputs = net(X)
        loss = criterion(outputs,y_mapped)
        #backward
        loss.backward()
        optimizer.step()

    with torch.no_grad():
      train_pred = net(train_X)
      train_correct = (train_pred.argmax(dim=1)==train_y).sum()
      train_acc = train_correct / len(train_pred)

      test_pred = net(test_X)
      test_correct = (test_pred.argmax(dim=1)==test_y).sum()
      test_acc = test_correct / len(test_pred)
      if test_acc_best<test_acc:
        test_acc_best = test_acc
        torch.save(net.state_dict(),PATH)


    print(f"Epoch {epoch+1}/{max_epochs}, Train accuracy: {100*train_acc:.2f}%")
    print(f"Epoch {epoch+1}/{max_epochs}, Test accuracy: {100*test_acc:.2f}%\n")

Epoch 1/10, Train accuracy: 83.77%
Epoch 1/10, Test accuracy: 83.40%

Epoch 2/10, Train accuracy: 87.34%
Epoch 2/10, Test accuracy: 86.27%

Epoch 3/10, Train accuracy: 89.55%
Epoch 3/10, Test accuracy: 87.87%

Epoch 4/10, Train accuracy: 92.37%
Epoch 4/10, Test accuracy: 89.53%

Epoch 5/10, Train accuracy: 92.51%
Epoch 5/10, Test accuracy: 88.80%

Epoch 6/10, Train accuracy: 93.77%
Epoch 6/10, Test accuracy: 90.30%

Epoch 7/10, Train accuracy: 95.51%
Epoch 7/10, Test accuracy: 90.87%

Epoch 8/10, Train accuracy: 96.34%
Epoch 8/10, Test accuracy: 91.47%

Epoch 9/10, Train accuracy: 95.69%
Epoch 9/10, Test accuracy: 91.13%

Epoch 10/10, Train accuracy: 96.79%
Epoch 10/10, Test accuracy: 91.73%



In [11]:
classes = ['car', 'dog', 'ship']
net_best = Net(3*32*32)
net_best.load_state_dict(torch.load(PATH))

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
  for data in testloader:
    images, labels = data
    labels = torch.tensor([class_mapping[label.item()] for label in labels]).to(device)
    test_outputs = net_best(images)
    _, test_preds = torch.max(test_outputs,1)
    for label,test_pred in zip(labels,test_preds):
      if label==test_pred:
        correct_pred[classes[label]]+=1
      total_pred[classes[label]]+=1

total_acc = 0
for classname, correct_count in correct_pred.items():
  acc = 100*float(correct_count)/total_pred[classname]
  total_acc+=acc
  print(f"Accuracy for class:{classname:5s} is {acc:.2f} %")

print(f"The overall test accuracy is {total_acc/3:.2f} %")


Accuracy for class:car   is 91.30 %
Accuracy for class:dog   is 96.20 %
Accuracy for class:ship  is 87.70 %
The overall test accuracy is 91.73 %



### Comparasion between two models

| Class  | Convolutional Layer Accuracy | Fully Connected Layer Accuracy | Percent Increase |
|--------|-----------------------------|---------------------------------|------------------|
| Car    | 91.30%                      | 82.40%                          | 10.80%           |
| Dog    | 96.20%                      | 89.70%                          | 6.89%            |
| Ship   | 87.70%                      | 80.40%                          | 9.11%            |
| Overall| 91.73%                      | 84.17%                          | 8.99%            |

As the table shows, the performance of convolutional layer increases around 9% over all.

> Indented block



### **P2.2 Shuffle pixels with convolutional layer**

In [54]:
def shuffle_pixels(image):
    flat_image = image.flatten()  # Flatten the image into a 1D array
    shuffled_pixels = np.random.permutation(flat_image)  # Shuffle the pixel values
    shuffled_image = shuffled_pixels.reshape(image.shape)  # Reshape back to the original image shape
    return shuffled_image

def filter_and_shuffle(dataset):
    shuffled_dataset = [(shuffle_pixels(ex[0]), ex[1]) for ex in dataset]
    return shuffled_dataset

In [56]:

trainset = filter_and_shuffle(trainset)
testset = filter_and_shuffle(testset)

In [58]:
trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size,shuffle=False)

train_full = DataLoader(trainset, batch_size=len(trainset),shuffle=True)
train_X, train_y = next(iter(train_full))
train_X, train_y = train_X.to(device),train_y.to(device)
train_y = torch.tensor([class_mapping[label.item()] for label in train_y]).to(device)


test_full = DataLoader(testset, batch_size=len(testset),shuffle=True)
test_X, test_y = next(iter(test_full))
test_X, test_y = test_X.to(device),test_y.to(device)
test_y = torch.tensor([class_mapping[label.item()] for label in test_y]).to(device)

In [59]:
net = Net(3*32*32).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr = 1e-3, momentum=0.9)
max_epochs = 10

train_acc_best = 0
test_acc_best = 0
PATH="/model.pt"

In [60]:
for epoch in range(max_epochs):

    for i,data in enumerate(trainloader):
        optimizer.zero_grad()
        X,y = data
        X = X.to(device)
        y = y.to(device)
        y_mapped = torch.tensor([class_mapping[label.item()] for label in y]).to(device)
        # forward
        outputs = net(X)
        loss = criterion(outputs,y_mapped)
        #backward
        loss.backward()
        optimizer.step()

    with torch.no_grad():
      train_pred = net(train_X)
      train_correct = (train_pred.argmax(dim=1)==train_y).sum()
      train_acc = train_correct / len(train_pred)

      test_pred = net(test_X)
      test_correct = (test_pred.argmax(dim=1)==test_y).sum()
      test_acc = test_correct / len(test_pred)
      if test_acc_best<test_acc:
        test_acc_best = test_acc
        torch.save(net.state_dict(),PATH)


    print(f"Epoch {epoch+1}/{max_epochs}, Train accuracy: {100*train_acc:.2f}%")
    print(f"Epoch {epoch+1}/{max_epochs}, Test accuracy: {100*test_acc:.2f}%\n")

Epoch 1/10, Train accuracy: 45.01%
Epoch 1/10, Test accuracy: 43.80%

Epoch 2/10, Train accuracy: 46.29%
Epoch 2/10, Test accuracy: 45.17%

Epoch 3/10, Train accuracy: 43.40%
Epoch 3/10, Test accuracy: 42.10%

Epoch 4/10, Train accuracy: 45.62%
Epoch 4/10, Test accuracy: 44.80%

Epoch 5/10, Train accuracy: 46.55%
Epoch 5/10, Test accuracy: 44.83%

Epoch 6/10, Train accuracy: 44.79%
Epoch 6/10, Test accuracy: 41.97%

Epoch 7/10, Train accuracy: 47.88%
Epoch 7/10, Test accuracy: 46.00%

Epoch 8/10, Train accuracy: 47.83%
Epoch 8/10, Test accuracy: 45.97%

Epoch 9/10, Train accuracy: 47.72%
Epoch 9/10, Test accuracy: 43.80%

Epoch 10/10, Train accuracy: 48.99%
Epoch 10/10, Test accuracy: 45.07%



In [61]:
classes = ['car', 'dog', 'ship']
net_best = Net(3*32*32)
net_best.load_state_dict(torch.load(PATH))

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
  for data in testloader:
    images, labels = data
    labels = torch.tensor([class_mapping[label.item()] for label in labels]).to(device)
    test_outputs = net_best(images)
    _, test_preds = torch.max(test_outputs,1)
    for label,test_pred in zip(labels,test_preds):
      if label==test_pred:
        correct_pred[classes[label]]+=1
      total_pred[classes[label]]+=1

total_acc = 0
for classname, correct_count in correct_pred.items():
  acc = 100*float(correct_count)/total_pred[classname]
  total_acc+=acc
  print(f"Accuracy for class:{classname:5s} is {acc:.2f} %")

print(f"The overall test accuracy is {total_acc/3:.2f} %")

Accuracy for class:car   is 64.10 %
Accuracy for class:dog   is 17.20 %
Accuracy for class:ship  is 56.70 %
The overall test accuracy is 46.00 %


### **P2.3 Fully connected layer with shuffled pixels**

In [64]:
class NetFC(nn.Module):
    def __init__(self,inputdim):
        super(NetFC,self).__init__()
        self.fc1 = nn.Linear(inputdim,512)
        self.fc2 = nn.Linear(512,3)

    def forward(self,x):
        x = torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x

net = NetFC(3*32*32).to(device)
optimizer = torch.optim.SGD(net.parameters(),lr = 1e-3, momentum=0.9)
max_epochs = 10

train_acc_best = 0
test_acc_best = 0
PATH="/model_fc.pt"

for epoch in range(max_epochs):

    for i,data in enumerate(trainloader):
        optimizer.zero_grad()
        X,y = data
        X = X.to(device)
        y = y.to(device)
        y_mapped = torch.tensor([class_mapping[label.item()] for label in y]).to(device)
        # forward
        outputs = net(X)
        loss = criterion(outputs,y_mapped)
        #backward
        loss.backward()
        optimizer.step()

    with torch.no_grad():
      train_pred = net(train_X)
      train_correct = (train_pred.argmax(dim=1)==train_y).sum()
      train_acc = train_correct / len(train_pred)

      test_pred = net(test_X)
      test_correct = (test_pred.argmax(dim=1)==test_y).sum()
      test_acc = test_correct / len(test_pred)
      if test_acc_best<test_acc:
        test_acc_best = test_acc
        torch.save(net.state_dict(),PATH)


    print(f"Epoch {epoch+1}/{max_epochs}, Train accuracy: {100*train_acc:.2f}%")
    print(f"Epoch {epoch+1}/{max_epochs}, Test accuracy: {100*test_acc:.2f}%\n")


classes = ['car', 'dog', 'ship']
net_best = Net(3*32*32)
net_best.load_state_dict(torch.load(PATH))

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
  for data in testloader:
    images, labels = data
    labels = torch.tensor([class_mapping[label.item()] for label in labels]).to(device)
    test_outputs = net_best(images)
    _, test_preds = torch.max(test_outputs,1)
    for label,test_pred in zip(labels,test_preds):
      if label==test_pred:
        correct_pred[classes[label]]+=1
      total_pred[classes[label]]+=1

total_acc = 0
for classname, correct_count in correct_pred.items():
  acc = 100*float(correct_count)/total_pred[classname]
  total_acc+=acc
  print(f"Accuracy for class:{classname:5s} is {acc:.2f} %")

print(f"The overall test accuracy is {total_acc/3:.2f} %")

Epoch 1/10, Train accuracy: 49.64%
Epoch 1/10, Test accuracy: 40.47%

Epoch 2/10, Train accuracy: 75.16%
Epoch 2/10, Test accuracy: 40.73%

Epoch 3/10, Train accuracy: 78.29%
Epoch 3/10, Test accuracy: 36.53%

Epoch 4/10, Train accuracy: 87.03%
Epoch 4/10, Test accuracy: 39.00%

Epoch 5/10, Train accuracy: 94.77%
Epoch 5/10, Test accuracy: 38.13%

Epoch 6/10, Train accuracy: 96.12%
Epoch 6/10, Test accuracy: 37.73%

Epoch 7/10, Train accuracy: 98.02%
Epoch 7/10, Test accuracy: 37.90%

Epoch 8/10, Train accuracy: 99.39%
Epoch 8/10, Test accuracy: 37.27%

Epoch 9/10, Train accuracy: 98.19%
Epoch 9/10, Test accuracy: 38.47%

Epoch 10/10, Train accuracy: 96.52%
Epoch 10/10, Test accuracy: 38.30%

Accuracy for class:car   is 39.00 %
Accuracy for class:dog   is 34.90 %
Accuracy for class:ship  is 48.30 %
The overall test accuracy is 40.73 %





| Class   | Convolutional Layer Accuracy | Fully Connected Layer Accuracy | Percent Increase |
|---------|-----------------------------|---------------------------------|------------------|
| Car     | 64.10%                      | 39.00%                          | 39.74%           |
| Dog     | 17.20%                      | 34.90%                          | -102.91%         |
| Ship    | 56.70%                      | 48.30%                          | 17.46%           |
| Overall | 46.00%                      | 40.73%                          | 12.86%           |


**Compare to normal dataset, the dataset with shuffled pixels:**

1. **The fully connected model**'s best train accuracy increase from 94% to 99.%. This is because shuffle pixels introduces more complexities to the model, allowing it to learn more information. In other words, it is overfitting.It performs very well in trainning data, but fails to generalize well in testing datasets.

2. For **convolutional model** both train and test accuracy drop a lot. This is because convolutional layers are designed to capture spatial features, it disrupt these spatial relations when we shuffling the pixels, resulting decrease both in train and test accuracy.